In [40]:
import pandas as pd

cols = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'label'
]

df = pd.read_csv('../kddcup.data', names=cols, index_col=False)

In [65]:
df.head()
attacks = ['back.', 'buffer_overflow.', 'ftp_write.', 'guess_passwd.', 'imap.', 'ipsweep.', 'probe.', 'land.', 'loadmodule.', 'multihop.', 'neptune.', 'nmap.', 'probe.', 'perl.', 'phf.', 'pod.', 'portsweep.', 'probe.', 'rootkit.', 'satan.', 'probe.', 'smurf.', 'spy.', 'teardrop.', 'warezclient.', 'warezmaster.']
data = df[(df['service'] == 'http') & (df['logged_in'] == 1)].copy()
data = data[['duration', 'src_bytes', 'dst_bytes', 'label']].copy()
data['label'] = data['label'] != 'normal.'

In [71]:
from sklearn import preprocessing
data['duration'] = (data['duration'] - data['duration'].mean()) / data['duration'].std()
data['src_bytes'] = (data['src_bytes'] - data['src_bytes'].mean()) / data['src_bytes'].std()
data['dst_bytes'] = (data['dst_bytes'] - data['dst_bytes'].mean()) / data['dst_bytes'].std()

In [73]:
data.head()
data.to_csv('evaluation_data.csv', index=False)

In [ ]:
eval_data = pd.read_csv('evaluation_data.csv', names=['duration', 'src_bytes', 'dst_bytes', 'label'], index_col=False)
output = []

N = 5
WINDOW_SIZE = 6000

num_detected = []
num_outliers = []

for i in range(eval_data.shape[0], step=WINDOW_SIZE):
    win_data = eval_data.iloc[i:i+WINDOW_SIZE,]
    num_true_outlier = win_data[win_data['label'] == 1].shape[0]
    o = min(N, num_true_outlier)
    
    num_true = 0
    
    for j in range(o):
        if win_data.iloc[output[j], 'label'] == 1:
            num_true += 1
            
    num_detected.append(num_true)
    num_outliers.append(o)